In [99]:
# --- Librerías estándar ---
import pandas as pd
import json
import time
import warnings
from datetime import datetime, timedelta
from pprint import pprint
# --- Configuración ---
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
# --- Librerías para manipulación de datos ---
import numpy as np

import requests as req
import swagger_client
from swagger_client.rest import ApiException
# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns
# --- Machine Learning ---
## Modelos
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
## Modelos de Scikit-Learn
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor, StackingClassifier
)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
## Preprocesamiento y selección de características
from sklearn.feature_selection import (
    RFE, RFECV, SelectFromModel, SelectKBest, f_classif
)
from sklearn.preprocessing import (
    LabelEncoder, MinMaxScaler, PowerTransformer, RobustScaler, StandardScaler
)
## Evaluación de modelos
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, 
    mean_squared_error, r2_score
)
## División de datos y optimización de hiperparámetros
from sklearn.model_selection import GridSearchCV, train_test_split
# --- Balanceo de datos ---
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN, SMOTE
from imblearn.under_sampling import NearMiss

In [100]:
df_total=pd.read_csv("/mnt/c/Users/danie/OneDrive/Desktop/Proyectos/proyecto_final/Notebooks/datos_climatologicos_limpios.csv")

In [101]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152262 entries, 0 to 152261
Data columns (total 60 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fecha              152262 non-null  object 
 1   indicativo         152262 non-null  object 
 2   nombre             152262 non-null  object 
 3   provincia          152262 non-null  object 
 4   altitud            152262 non-null  int64  
 5   tmed               152262 non-null  float64
 6   prec               152262 non-null  float64
 7   tmin               152262 non-null  float64
 8   tmax               152262 non-null  float64
 9   velmedia           152262 non-null  float64
 10  sol                48224 non-null   float64
 11  presMax            152262 non-null  float64
 12  presMin            152262 non-null  float64
 13  hrMedia            152262 non-null  float64
 14  racha              152262 non-null  float64
 15  hrMax              152262 non-null  float64
 16  hr

In [ ]:
#procedemos a seleccionar las semanas en las que se puede sembrar cereales en el 2025 y 2026

In [92]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Definir el rango de fechas
start_date = '2025-01-01'
end_date = '2026-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Crear un DataFrame con las fechas
df_prediccion = pd.DataFrame(date_range, columns=['fecha'])

# Extraer características de la fecha
df_prediccion['anio'] = df_prediccion['fecha'].dt.year
df_prediccion['mes'] = df_prediccion['fecha'].dt.month
df_prediccion['semana'] = df_prediccion['fecha'].dt.isocalendar().week


df_prediccion_fecha = df_prediccion[['fecha']].copy()

columnas_X = ['altitud', 'prec', 'semana', 'anio', 'mes', 'racha_log', 'nombre_encoded', '620', '667', 'tmax_robust', 'tmed_robust', 'prec_anual_yeo', 'sol_def']

# Extraer X 
X = df_total[columnas_X]
y = df_total['cereales']

# 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

# Entrenar el modelo XGBoost
modelo = xgb.XGBClassifier(scale_pos_weight=len(y_train_res) / sum(y_train_res == 1), random_state=42)
modelo.fit(X_train_res, y_train_res)


for col in columnas_X:
    if col not in df_prediccion.columns:
        df_prediccion[col] = 0  # Rellenar con ceros o valores más adecuados si los tienes

# Ordenar las columnas en el mismo orden que en X_train
df_prediccion = df_prediccion[columnas_X]

# Escalar los datos
df_prediccion_scaled = scaler.transform(df_prediccion)

# Hacer predicción
df_prediccion['cereales'] = modelo.predict(df_prediccion_scaled)

# Agregar la columna 'nombre' desde df_total (sin repetir estaciones)
df_nombres = df_total[['nombre']].drop_duplicates().reset_index(drop=True)

# Crear todas las combinaciones posibles de 'nombre' con fechas del rango
df_nombres['key'] = 1
df_prediccion['key'] = 1
df_secano = df_prediccion.merge(df_nombres, on='key').drop(columns=['key'])

# Seleccionar solo las columnas finales
df_secano = df_secano[['nombre', 'anio', 'mes', 'semana', 'cereales']]

# Mostrar el resultado
print(df_secano)

                       nombre  anio  mes  semana  cereales
0       PUERTO DE NAVACERRADA  2025    1       1         0
1                      GETAFE  2025    1       1         0
2              MADRID, RETIRO  2025    1       1         0
3      MADRID, CUATRO VIENTOS  2025    1       1         0
4              COLMENAR VIEJO  2025    1       1         0
...                       ...   ...  ...     ...       ...
15325         ARGANDA DEL REY  2026   12      53         0
15326       ALCALA DE HENARES  2026   12      53         0
15327    ROZAS DE PUERTO REAL  2026   12      53         0
15328               ALPEDRETE  2026   12      53         0
15329                 TIELMES  2026   12      53         0

[15330 rows x 5 columns]


In [102]:
#legumbres 2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import optuna

# Verificar tamaños iniciales
print("Tamaño de df_total:", df_total.shape)
print("Tamaño de df_secano:", df_secano.shape)

# 📌 1. Preparación de datos históricos
columnas_numericas = ['altitud', 'prec', 'presMax', 'presMin', 'hrMedia', 'semana', 'anio', 'mes',
                      'racha_log', 'nombre_encoded', '533', '540', '594', '605', '609', '620', 
                      '665', '667', '672', '690', '740', '763', '884', '890', '924', '1004', 
                      '1030', '1159', '1450', '1532', '1893', 'prec_log', 'tmin_robust', 
                      'tmax_robust', 'tmed_robust', 'hrMax_box', 'hrMin_robust', 'velmedia_log', 
                      'prec_anual_yeo', 'sol_def', 'soldef_yeo']

X = df_total[columnas_numericas]
y = df_total['legumbres']

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 📌 2. Balanceo con SMOTE focalizado
smote = SMOTE(sampling_strategy=0.5, k_neighbors=3, random_state=42)  # Relación 2:1
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
print("Tamaño de X_train después de SMOTE:", X_train_balanced.shape)

# 📌 3. Definición de la función objetivo para Optuna
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 5, 30),  # Peso para clase 1
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10.0),
        'n_jobs': -1  # Usar todos los hilos
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test_scaled)
    f1_score_1 = classification_report(y_test, y_pred, output_dict=True)['1']['f1-score']
    return f1_score_1

# 📌 4. Optimización con Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # 50 iteraciones para una buena búsqueda
print("Mejores hiperparámetros:", study.best_params)
print("Mejor F1-score (1):", study.best_value)

# 📌 5. Entrenamiento del modelo final con los mejores parámetros
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'binary_logloss'
best_params['boosting_type'] = 'gbdt'
best_params['n_jobs'] = -1

lgbm_model = lgb.LGBMClassifier(**best_params)
print("\nEntrenando LightGBM con mejores parámetros...")
lgbm_model.fit(X_train_balanced, y_train_balanced)

# Evaluación
y_pred = lgbm_model.fit(X_train_balanced, y_train_balanced).predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
print(f"LightGBM - Accuracy: {accuracy:.4f}, F1-score (1): {report['1']['f1-score']:.4f}")
print("\nReporte de clasificación completo:")
print(classification_report(y_test, y_pred))

# 📌 6. Importancia de características
feature_importance = pd.DataFrame({
    'feature': columnas_numericas,
    'importance': lgbm_model.feature_importances_
}).sort_values('importance', ascending=False)
print("\nImportancia de características:")
print(feature_importance.head(10))

# 📌 7. Preparación de df_secano para predicción
df_secano_pred = df_secano[df_secano['anio'].isin([2025, 2026])].copy()

# Calcular promedios de características meteorológicas por 'nombre' desde df_total
df_caracteristicas = df_total.groupby('nombre')[columnas_numericas].mean().reset_index()

# Unir con df_secano_pred
df_secano_pred = df_secano_pred.merge(df_caracteristicas, on='nombre', how='left')

# Asegurar que todas las columnas de columnas_numericas estén presentes
for col in columnas_numericas:
    if col not in df_secano_pred.columns:
        df_secano_pred[col] = 0
    if col in ['semana', 'anio', 'mes'] and col in df_secano.columns:
        df_secano_pred[col] = df_secano[df_secano['anio'].isin([2025, 2026])][col]

# Preparar X_futuro y escalar
X_futuro = df_secano_pred[columnas_numericas]
X_futuro_scaled = scaler.transform(X_futuro)

# 📌 8. Predicción con el modelo LightGBM
predicciones = lgbm_model.predict(X_futuro_scaled)

# Agregar predicciones a df_secano como 'legumbres2'
df_secano.loc[df_secano['anio'].isin([2025, 2026]), 'legumbres'] = predicciones

# 📌 9. Verificación
print("\n📌 df_secano con predicciones de legumbres (primeras 5 filas de 2025-2026):")
print(df_secano[df_secano['anio'].isin([2025, 2026])][['nombre', 'anio', 'mes', 'semana', 'cereales', 'legumbres']].head())
print("\nDistribución de predicciones para legumbres:")
print(df_secano[df_secano['anio'].isin([2025, 2026])]['legumbres'].value_counts(normalize=True))

Tamaño de df_total: (152262, 60)
Tamaño de df_secano: (15330, 6)


[I 2025-03-04 19:15:12,337] A new study created in memory with name: no-name-6eb779cf-769a-43cb-a105-0a23cff42c1d


Tamaño de X_train después de SMOTE: (180453, 41)


/tmp/ipykernel_138761/4234331504.py:47: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.1),
/tmp/ipykernel_138761/4234331504.py:50: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
/tmp/ipykernel_138761/4234331504.py:51: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
/tmp/ipykernel_138761/4234331504.py:53: FutureWarning: suggest_loguniform has been deprecat

Mejores hiperparámetros: {'num_leaves': 96, 'max_depth': 13, 'learning_rate': 0.0849690239517885, 'n_estimators': 840, 'min_child_samples': 26, 'subsample': 0.8549741577692929, 'colsample_bytree': 0.8408921418083494, 'scale_pos_weight': 8, 'reg_alpha': 2.153466932414961e-05, 'reg_lambda': 0.035654051480587236}
Mejor F1-score (1): 0.8204456094364351

Entrenando LightGBM con mejores parámetros...
LightGBM - Accuracy: 0.9955, F1-score (1): 0.8204

Reporte de clasificación completo:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30084
           1       0.79      0.85      0.82       369

    accuracy                           1.00     30453
   macro avg       0.90      0.92      0.91     30453
weighted avg       1.00      1.00      1.00     30453


Importancia de características:
           feature  importance
5           semana        9149
6             anio        7383
38  prec_anual_yeo        6877
4          hrMedia        5112
8

In [103]:
df_secano[(df_secano.legumbres==1)][['nombre','anio','mes','semana','legumbres']]

,nombre,anio,mes,semana,legumbres
697,COLMENAR VIEJO,2025,2,6,1
718,COLMENAR VIEJO,2025,2,6,1
739,COLMENAR VIEJO,2025,2,6,1
760,COLMENAR VIEJO,2025,2,6,1
781,COLMENAR VIEJO,2025,2,6,1
...,...,...,...,...,...
11466,PUERTO DE NAVACERRADA,2026,7,27,1
11487,PUERTO DE NAVACERRADA,2026,7,27,1
11508,PUERTO DE NAVACERRADA,2026,7,27,1
11529,PUERTO DE NAVACERRADA,2026,7,27,1


In [104]:
df_secano[(df_secano.cereales==1)][['nombre','anio','mes','semana','cereales']]

,nombre,anio,mes,semana,cereales


In [105]:
df_secano.to_csv("df_secano_final.csv", index=False)
# df_total.to_csv("prediccion25_26", index=False)